In [3]:
import paramiko

# 서버 접속 정보
ip = "172.16.254.12"
user = "root"
pw = "asd123!@"

# 고정된 경로
HTTPD_CONF = "/etc/apache2/sites-available/000-default.conf"
VHOST_DIR = "/etc/apache2/sites-available"
APACHE2_CONF = "/etc/apache2/apache2.conf"

# 동적으로 받을 값
WEB_DOCUMENT_ROOT = ""
VHOST_CONF = ""
SERVER_NAME = ""
ERROR_LOG = ""
CUSTOM_LOG = ""


def cc(cmd):
    """SSH로 명령어를 실행하고 결과 출력"""
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()


def set_ufw():
    """방화벽 상태 확인 및 중지"""
    print("🔥 방화벽 설정 중...")
    cmd = """
systemctl stop ufw
systemctl disable ufw
echo 'UFW 방화벽 중지 완료'
"""
    cc(cmd)


def install_package():
    """Apache 설치"""
    print("📦 Apache 설치 중...")
    cmd = """
apt update -y
apt install -y apache2
"""
    cc(cmd)


def start_apache():
    """Apache 실행"""
    print("🚀 Apache 실행 중...")
    cmd = """
systemctl enable apache2
systemctl restart apache2
systemctl status apache2 --no-pager
"""
    cc(cmd)


def set_user_dir_and_vhost_conf():
    """사용할 웹 디렉토리 입력"""
    global WEB_DOCUMENT_ROOT
    WEB_DOCUMENT_ROOT = input("사용할 웹 디렉토리를 입력하세요 (예: /home/team1 또는 /var/www/young): ").strip()
    cc(f"mkdir -p {WEB_DOCUMENT_ROOT} && chmod 755 {WEB_DOCUMENT_ROOT}")
    print(f"✅ 웹 문서 루트: {WEB_DOCUMENT_ROOT}")

def set_apache2_conf() :
    cmd = f"""
    sed -i 's|<Directory /var/www/>|<Directory {WEB_DOCUMENT_ROOT}>|' {APACHE2_CONF}
    """
    cc(cmd)

def set_vhostconf():
    """가상 호스트 설정 파일 생성"""
    global VHOST_CONF, SERVER_NAME, ERROR_LOG, CUSTOM_LOG

    vhost_name = input("VHost 설정 파일을 만들겠습니다 원하는 이름을 입력하세요 (예: team1.conf): ")
    VHOST_CONF = f"{VHOST_DIR}/{vhost_name}"

    SERVER_NAME = input("ServerName을 입력하세요 (예: test.local): ")
    ERROR_LOG = input("ErrorLog 경로를 입력하세요 (예: /var/log/apache2/team1_error.log): ")
    CUSTOM_LOG = input("CustomLog 경로를 입력하세요 (예: /var/log/apache2/team1_access.log): ")

    print(f" {VHOST_CONF} 생성 중...")

    cmd = f"""
cat <<EOF > {VHOST_CONF}
<VirtualHost *:80>
    ServerAdmin webmaster@localhost
    ServerName {SERVER_NAME}
    DocumentRoot {WEB_DOCUMENT_ROOT}

    <Directory {WEB_DOCUMENT_ROOT}>
        AllowOverride None
        Options Indexes FollowSymLinks
        Require all granted
    </Directory>

    ErrorLog {ERROR_LOG}
    CustomLog {CUSTOM_LOG} combined
</VirtualHost>
EOF

a2ensite {vhost_name}
a2dissite 000-default.conf
systemctl reload apache2
mkdir -p {ERROR_LOG}; mkdir -p {CUSTOM_LOG};
echo ' VirtualHost 설정 완료: {VHOST_CONF}'
"""
    cc(cmd)


def mk_index():
    """기본 index.html 생성"""
    print("📄 index.html 생성 중...")
    cmd = f"echo '<h1>This is index.html for Test by mk_index func</h1>' > {WEB_DOCUMENT_ROOT}/index.html"
    print(f"{WEB_DOCUMENT_ROOT} 경로에 파일이 생성되었습니다")
    cc(cmd)


def restart_apa():
    """Apache 재시작"""
    print("🔁 Apache 재시작 중...")
    cc("systemctl restart apache2 && echo 'Apache 재시작 완료'")

def install_ftp():
    cc("apt install -y vsftpd")

def start_ftp():
    cc("systemctl enable vsftpd; systemctl start vsftpd")

def go():
    """전체 자동화 실행"""
    set_ufw()
    install_package()
    start_apache()
    set_user_dir_and_vhost_conf()
    set_apache2_conf()
    set_vhostconf()
    mk_index()
    restart_apa()
    print("🎉 모든 작업이 완료되었습니다!")

    install_ftp()
    start_ftp()
if __name__ == "__main__":
    go()


🔥 방화벽 설정 중...
Synchronizing state of ufw.service with SysV service script with /usr/lib/systemd/systemd-sysv-install.
Executing: /usr/lib/systemd/systemd-sysv-install disable ufw
Removed "/etc/systemd/system/multi-user.target.wants/ufw.service".
UFW 방화벽 중지 완료

📦 Apache 설치 중...
Hit:1 http://kr.archive.ubuntu.com/ubuntu noble InRelease
Hit:2 http://kr.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:3 http://kr.archive.ubuntu.com/ubuntu noble-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
1 package can be upgraded. Run 'apt list --upgradable' to see it.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apache2-bin apache2-data apache2-utils libapr1t64 libaprutil1-dbd-sqlite3
  libaprutil1-ldap libaprutil1t64 liblua5.4-0
Suggested packages:
  apache2-

사용할 웹 디렉토리를 입력하세요 (예: /home/team1 또는 /var/www/young):  /home/test



✅ 웹 문서 루트: /home/test



VHost 설정 파일을 만들겠습니다 원하는 이름을 입력하세요 (예: team1.conf):  vhost.conf
ServerName을 입력하세요 (예: test.local):  test.local
ErrorLog 경로를 입력하세요 (예: /var/log/apache2/team1_error.log):  /var/log/apache2/test_error.log
CustomLog 경로를 입력하세요 (예: /var/log/apache2/team1_access.log):  /var/log/apache2/test_access.log


 /etc/apache2/sites-available/vhost.conf 생성 중...
Enabling site vhost.
To activate the new configuration, you need to run:
  systemctl reload apache2
Site 000-default disabled.
To activate the new configuration, you need to run:
  systemctl reload apache2
mkdir: cannot create directory ‘/var/log/apache2/test_error.log’: File exists
mkdir: cannot create directory ‘/var/log/apache2/test_access.log’: File exists
 VirtualHost 설정 완료: /etc/apache2/sites-available/vhost.conf

📄 index.html 생성 중...
/home/test 경로에 파일이 생성되었습니다

🔁 Apache 재시작 중...
Apache 재시작 완료

🎉 모든 작업이 완료되었습니다!
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
vsftpd is already the newest version (3.0.5-0ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.

Synchronizing state of vsftpd.service with SysV service script with /usr/lib/systemd/systemd-sysv-install.
Executing: /usr/lib/systemd/systemd-sysv-install enable vsftpd

